## 워크플로우 요약

[선행작업] KoBART로 기사 본문의 [생성 요약문]을 미리 준비합니다.  
[데이터 구축] [정상 제목]+[생성 요약문] / [낚시성 제목] + [생성 요약문]을 페어로 묶어 대조 학습용 데이터셋을 만듭니다.  
[모델 학습] KoSimCSE 모델을 ContrastiveLoss로 파인튜닝합니다.  
[평가] 학습된 모델로 유사도를 계산하고 임계값(Threshold)을 설정합니다.

## 1단계: 선행 작업 - KoBART 요약 모델 (가정)

본 노트북은 1단계가 이미 완료되었다고 가정합니다.
(보유한 <요약문 생성 데이터>(B)로 KoBART 모델이 이미 파인튜닝되었습니다.)

이 KoBART 모델을 사용하여, <낚시성 뉴스 데이터>(A)의 모든 본문에 대한 [생성된 요약문]을 확보했습니다.

## 2단계: KoSimCSE 학습용 데이터셋 구축
KoSimCSE 모델을 '지도 학습(Supervised)' 방식의 대조 학습으로 파인튜닝하기 위한 데이터셋을 구성합니다.

Positive Pair (일치, 레이블 1.0)
입력 1 (text1): <낚시성 뉴스 데이터>(A.i)의 [정상 기사 제목]
입력 2 (text2): 1단계에서 생성한 해당 기사의 [생성된 요약문]

Negative Pair (불일치, 레이블 0.0)
입력 1 (text1): <낚시성 뉴스 데이터>(A.ii)의 [일치하지 않는 기사 제목]
입력 2 (text2): 1단계에서 생성한 해당 기사의 [생성된 요약문]

**현재 중간제출 코드에서는 요약문 모델이 준비 되지 않아 '요약문' 부분을 데이터의 본문으로 치환하여 테스트하고   
그 결과를 바탕으로 모델을 제작하였습니다**

In [ ]:

#-- 데이터 준비, 처음 한번만 하면 되니간 전체 주석처리!

# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/gdrive')
print("Google Drive 마운트 완료.")

# 2. (NEW) CSV 파일 경로 확인 (예시)
# 실제 데이터 로드는 8번 셀(f3cee925)에서 이뤄집니다.
import os

# ★★★ 사용자님이 말씀하신 CSV 파일의 전체 경로를 여기에 입력하세요. ★★★
csv_path = "/content/gdrive/My Drive/Colab_Notebooks/ssu_NLP/new_summarization30000.csv"
print(f"데이터로 사용할 CSV 파일 경로 (예상): {csv_path}")

if not os.path.exists(csv_path):
    print("--- 🔴 경고 ---")
    print("CSV 파일을 찾을 수 없습니다. 8번 셀의 'csv_path'를 정확히 수정해주세요.")
else:
    print("CSV 파일이 해당 경로에 존재하는 것을 확인했습니다.")

# --- 기존 압축 해제 로직은 주석 처리 ---
# print("기존 ZIP 압축 해제 로직은 new_summarization30000.csv 파일을 사용하므로 건너뜁니다.")


Mounted at /content/gdrive
Google Drive 마운트 완료.
데이터로 사용할 CSV 파일 경로 (예상): /content/gdrive/My Drive/Colab_Notebooks/ssu_NLP/new_summarization30000.csv
CSV 파일이 해당 경로에 존재하는 것을 확인했습니다.


## 3단계: KoSimCSE 모델 구성 및 파인튜닝
sentence-transformers 라이브러리를 사용하여 모델을 구성하고 학습시킵니다.

In [ ]:
#-- 3.1. 필요 라이브러리 설치 (Jupyter/Colab 환경)
'''
%pip install sentence-transformers torch

#기타 필요 라이브러리
%pip install datasets
%pip install accelerate
'''

'\n%pip install sentence-transformers torch\n\n#기타 필요 라이브러리\n%pip install datasets\n%pip install accelerate\n'

In [ ]:
# 3.2. 모듈 임포트
import os
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import torch


print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [ ]:
# 3.3. 학습 데이터 정의 (★CSV 파일에서 로드 및 분리★)
# new_summarization30000.csv 파일에서 (text1, text2, label) 형식의 데이터를 생성합니다.

# 1. 필요한 모듈 임포트
import random
import os
import pandas as pd # (NEW) Pandas 임포트

# 2. (NEW) 데이터셋 경로 설정
# ★★★ 이 경로는 4번 셀에서 확인한 경로와 일치해야 합니다. ★★★
csv_path = "/content/gdrive/My Drive/Colab_Notebooks/ssu_NLP/new_summarization30000.csv"

# 3. (NEW) all_data_pairs 생성 (Pandas 로직)
all_data_pairs = []
total_rows_processed = 0

print(f"CSV 파일 로드 시도: {csv_path}")

try:
    # 3.1. CSV 파일 로드
    df = pd.read_csv(csv_path)
    print(f"CSV 로드 완료. 총 {len(df)}개의 행 발견.")

    # (NEW) 3.2. 사용할 컬럼(newsTitle, summary, clickbaitClass)에 결측치가 있는 행 제거
    df = df.dropna(subset=['newsTitle', 'summary', 'clickbaitClass'])
    print(f"결측치 제거 후 유효한 행: {len(df)}개")

    # 3.3. 데이터프레임 순회
    # (이미지에서 확인한 컬럼명: 'newsTitle', 'summary', 'clickbaitClass' 사용)
    for row in df.to_dict('records'):

        title = row.get('newsTitle')
        summary_text = row.get('summary')
        class_label = row.get('clickbaitClass') # (0 또는 1)

        # (Pandas는 결측치를 NaN (float)으로 다룸)
        def is_str_valid(s):
            return s and isinstance(s, str) and s.strip()

        # 제목과 요약문이 유효한 문자열인지 확인
        if is_str_valid(title) and is_str_valid(summary_text):
            try:
                # class_label이 0, 1, "0", "1", 0.0 등 어떤 형태든 int로 변환
                class_val_int = int(float(class_label))

                if class_val_int == 0:
                    # 0 = 정상 기사 -> Positive Pair (Label 1.0)
                    final_label = 1.0
                elif class_val_int == 1:
                    # 1 = 낚시성 기사 -> Negative Pair (Label 0.0)
                    final_label = 0.0
                else:
                    continue # 0이나 1이 아니면 건너뜀

                # (제목, 요약문, 낚시성_여부_레이블)
                all_data_pairs.append((title, summary_text, final_label))
                total_rows_processed += 1

            except (ValueError, TypeError):
                # int 변환 실패 시 (예: class_label이 이상한 값일 경우)
                continue

except FileNotFoundError:
    print(f"--- 🔴 오류: {csv_path} 에서 파일을 찾을 수 없습니다. ---")
    print("--- 4번 셀(drive.mount)이 실행되었는지, 파일 경로가 올바른지 확인하세요. ---")
    df = pd.DataFrame() # 빈 데이터프레임
except KeyError as e:
    print(f"--- 🔴 오류: CSV에 필요한 컬럼이 없습니다 ---")
    print(f"'{e.args[0]}' 컬럼을 찾을 수 없습니다. (newsTitle, summary, clickbaitClass 중 하나)")
except Exception as e:
    print(f"CSV 로드 또는 처리 중 예기치 않은 오류 발생: {e}")


# (기존 로직) 4. 데이터 셔플 및 분리
if all_data_pairs:
    random.shuffle(all_data_pairs) # ★전체 데이터를 1회 셔플

    # (NEW) 90%는 학습, 10%는 검증(Validation)용으로 분리
    validation_split_ratio = 0.1
    split_index = int(len(all_data_pairs) * (1 - validation_split_ratio))

    train_data_list = all_data_pairs[:split_index]
    validation_data_list = all_data_pairs[split_index:] # 12번 셀에서 사용할 검증 데이터

    print(f"\n--- 데이터 로드 및 분리 완료 ---")
    print(f"  성공적으로 처리한 CSV 행 수: {total_rows_processed} 개")
    print(f"  생성된 총 학습 페어(쌍): {len(all_data_pairs)} 개")
    print(f"  - (학습용)  train_data_list: {len(train_data_list)} 개 (90%)")
    print(f"  - (검증용) validation_data_list: {len(validation_data_list)} 개 (10%)")

else:
    print("경고: 학습 데이터를 하나도 로드하지 못했습니다. 파일 경로와 CSV 컬럼명을 확인하세요.")
    train_data_list = []
    validation_data_list = []

print(f"\n총 {len(train_data_list)}개의 학습 예시 데이터 준비 완료")

CSV 파일 로드 시도: /content/gdrive/My Drive/Colab_Notebooks/ssu_NLP/new_summarization30000.csv
CSV 로드 완료. 총 30003개의 행 발견.
결측치 제거 후 유효한 행: 30003개

--- 데이터 로드 및 분리 완료 ---
  성공적으로 처리한 CSV 행 수: 30003 개
  생성된 총 학습 페어(쌍): 30003 개
  - (학습용)  train_data_list: 27002 개 (90%)
  - (검증용) validation_data_list: 3001 개 (10%)

총 27002개의 학습 예시 데이터 준비 완료


In [ ]:
# 3.4.0. 필요한 모듈 추가 임포트
from sentence_transformers import models

# 3.4.1. 사용할 KoSimCSE 모델 이름 (사용자님이 찾으신 모델)
model_name = 'BM-K/KoSimCSE-roberta-multitask'

# 3.4.2. CUDA 사용 가능 여부 체크
device = "cuda" if torch.cuda.is_available() else "cpu"

# 3.4.3. SentenceTransformer 모델 수동 조립
try:
    # 1. '엔진' (기본 Transformer 모델) 로드
    word_embedding_model = models.Transformer(model_name)

    # 2. '풀링 레이어' 정의
    # SimCSE는 보통 [CLS] 토큰의 임베딩을 문장 벡터로 사용합니다.
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode='cls' # 'mean'이 아닌 'cls'로 명시적 지정
    )

    # 3. 두 모듈을 결합하여 최종 SentenceTransformer 모델 생성
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device=device)

    print(f"모델 '{model_name}' 로드 및 '[CLS] 풀링' 적용 완료.")
    print(f"사용 장치: {device}")

except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    print("Hugging Face Hub에 연결할 수 있는지, 모델 이름이 올바른지 확인해주세요.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

모델 'BM-K/KoSimCSE-roberta-multitask' 로드 및 '[CLS] 풀링' 적용 완료.
사용 장치: cuda


In [ ]:
# 3.5. 학습 데이터셋 형식 변환 (InputExample)
# sentence-transformers 라이브러리의 ContrastiveLoss는 InputExample 형식의 데이터를 사용합니다.
# 여기서 texts 리스트에는 [text1, text2]가, label에는 0.0 또는 1.0이 들어갑니다.

# (이전 3.3. 셀에서 train_data_list가 정의되었다고 가정합니다.)

train_examples = []
for data in train_data_list:
    text1 = data[0]
    text2 = data[1]
    label = float(data[2])
    train_examples.append(InputExample(texts=[text1, text2], label=label))

print(f"총 {len(train_examples)}개의 InputExample 생성 완료")

# 첫 번째 예시 확인
if train_examples:
    print("\n--- 첫 번째 학습 예시 --- ")
    print(f"Text 1: {train_examples[0].texts[0]}")
    print(f"Text 2: {train_examples[0].texts[1]}")
    print(f"Label: {train_examples[0].label}")

#--
# Text 1: 기사의 **'헤드라인' (제목)**입니다. --> (Label이 1.0이면 newsTitle - 정상 제목) / (Label이 0.0이면 newTitle - 낚시성 제목)
# Text 2: 기사의 **'본문' (newsContent)**입니다. (현재는 원문 전체이고, 7번 셀을 다시 실행하면 512글자로 잘린 본문이 됩니다.)

# 출력된 예시(Label: 0.0)는 이 두 텍스트가 서로 **'일치하지 않는 낚시성 기사 쌍'**이라는 것을 의미합니다.

총 27002개의 InputExample 생성 완료

--- 첫 번째 학습 예시 --- 
Text 1: 무섭게 치솟는 美 임대료…맨해튼 아파트 월세 중위값 '502만원'
Text 2: 미국의 주택 임대료가 가파르게 상승하는 가운데 맨해튼 지역 아파트 월세 중위가격이 처음으로 4000달러(약 502만원)을 돌파했다고 블룸버그 통신이 8일(현지시간) 밝혔는데, 이 중위값은 아파트 매매가격을 일렬로 세웠을 때 정중앙에 위치하는 가격을 의미한다.
Label: 0.0


In [ ]:
#-- 3.6.a 학습 설정 (DataLoader 및 Contrastive Loss)
# DataLoader: 학습 데이터를 배치(batch) 단위로 모델에 전달합니다.
# ContrastiveLoss: 프로젝트 계획 5.B에서 제안한 대조 학습 손실 함수입니다.
# label=1.0 (Positive) 페어는 임베딩 거리를 가깝게 (0에 가깝게) 만듭니다.
# label=0.0 (Negative) 페어는 임베딩 거리를 margin 값보다 멀게 만듭니다.


# DataLoader 생성
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32) # 2 -> 32로 변경

# Loss 함수 정의: ContrastiveLoss
# margin: Negative Pair가 가져야 할 최소 거리 (기본값 0.5)
train_loss = losses.ContrastiveLoss(model=model, margin=0.5)

print("DataLoader 및 ContrastiveLoss 정의 완료")



#-- 3.6.b. 검증(Validation) 데이터셋 및 평가자(Evaluator) 준비 ---
from sentence_transformers.evaluation import BinaryClassificationEvaluator
import random

# (이전 3.5 셀의 train_examples가 정의되었다고 가정합니다)

# --- [ ★ 수정된 부분 ★ ] ---
# (8번 셀에서 미리 분리해 둔 'validation_data_list'를 사용합니다)
if 'validation_data_list' in locals() and validation_data_list:

    print(f"미리 분리된 검증(validation) 세트 {len(validation_data_list)}개를 사용합니다.")

    # 평가자가 요구하는 형식으로 분리
    val_sentences1 = [data[0] for data in validation_data_list]
    val_sentences2 = [data[1] for data in validation_data_list]
    val_labels = [data[2] for data in validation_data_list]

    # BinaryClassificationEvaluator: (문장1, 문장2, 레이블[0/1]) 형식에 사용
    evaluator = BinaryClassificationEvaluator(
        val_sentences1,
        val_sentences2,
        val_labels,
        name='validation'
    )
    print("평가자(evaluator) 준비 완료.")
else:
    print("검증 세트(validation_data_list)를 찾을 수 없습니다. evaluator 없이 진행합니다.")
    evaluator = None
# --- [ ★ 수정된 부분 ★ ] ---

DataLoader 및 ContrastiveLoss 정의 완료
미리 분리된 검증(validation) 세트 3001개를 사용합니다.
평가자(evaluator) 준비 완료.


In [ ]:
#-- 3.7. 모델 학습 실행
# 이제 model.fit()을 호출하여 파인튜닝을 시작합니다.
# (NEW) Google Drive의 고유한 경로에 최고 성능 모델이 저장됩니다.

# (NEW) 1. 로깅 모듈 임포트
import logging
import time
import os     # (NEW) os 모듈 임포트
from datetime import datetime # (NEW) 날짜/시간 임포트

# (NEW) 2. 로깅 설정 (INFO 레벨)
# model.fit()이 INFO 레벨의 로그를 출력하도록 설정합니다.
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

# (NEW) 3. 'wandb' 로깅 강제 비활성화 (API 키 프롬프트 방지)
os.environ["WANDB_DISABLED"] = "true"

# 학습 파라미터 설정
num_epochs = 5 # 에포크 (실제 대규모 데이터에서는 1~3 사이로 조절)

# (NEW) warmup_steps를 실제 스텝 수에 맞게 조정 (예: 전체 스텝의 5%)
# (10번 셀에서 len(train_dataloader)가 정의되었다고 가정)
try:
    total_steps = len(train_dataloader) * num_epochs
    # warmup_steps는 최소 100 이상으로 설정하는 것이 안정적입니다.
    warmup_steps_count = max(100, int(total_steps * 0.05))
    print(f"총 스텝 수: {total_steps}, Warmup 스텝 수: {warmup_steps_count} (5%~)")
except NameError:
    print("오류: train_dataloader가 정의되지 않았습니다. 10번 셀을 먼저 실행하세요.")
    warmup_steps_count = 100 # 임의의 값으로 설정
    total_steps = 0

# --- [ ★★★ (통합) 동적 저장 경로 생성 ★★★ ] ---
# 1. 기본 저장 폴더 (사용자님이 요청하신 경로)
# (주의: Colab의 'My Drive'는 'MyDrive'로 띄어쓰기 없이 붙여 써야 합니다)
base_save_dir = "/content/gdrive/MyDrive/Colab_Notebooks/ssu_NLP/Trained_simcse_model"

# 2. (NEW) 현재 시간을 기반으로 _YYMMDD_HHMM 형식의 타임스탬프 생성
#    (예: _251112_1841)
current_time_str = datetime.now().strftime("_%y%m%d_%H%M")

# 3. (NEW) 최종 모델 폴더명 (예: model_251112_1841)
model_folder_name = f"model{current_time_str}"

# 4. 최종 저장 경로
model_save_path = os.path.join(base_save_dir, model_folder_name)

# 5. (NEW) 기본 폴더가 없으면 생성
os.makedirs(base_save_dir, exist_ok=True)
# --- [ ★★★ (통합) 수정 끝 ★★★ ] ---


print(f"모델 파인튜닝 시작... (Epochs: {num_epochs})")
print(f"★★★ (중요) 최고 성능 모델이 Google Drive 경로에 저장됩니다: {model_save_path} ★★★")

# (이전 셀에서 evaluator가 생성되었다고 가정)
if 'evaluator' in locals() and evaluator:
    print("검증자(evaluator)가 감지되었습니다. 매 에포크(Epoch) 종료 시 검증을 수행합니다.")
else:
    print("검증자(evaluator)가 없어, 에포크가 끝날 때만 로깅됩니다.")


# (NEW) 학습 시간 측정을 위한 시작 시간 기록
start_time = time.time()

# 모델 파인튜닝
# (3.4의 'model' 변수와 3.6의 'train_loss' 변수를 사용)
model.fit(train_objectives=[(train_dataloader, train_loss)],

          # --- [ ★★★ (통합) 수정된 부분 ★★★ ] ---
          epochs=num_epochs,               # '진짜' 에포크 사용
          evaluator=evaluator if 'evaluator' in locals() else None, # 이전 셀에서 만든 평가자
          save_best_model=True,          # (NEW) ★True로 변경★: 최고 성능 모델 저장
          # --- [ ★★★ (통합) 수정 끝 ★★★ ] ---

          warmup_steps=warmup_steps_count,
          output_path=model_save_path, # 이 경로가 Google Drive의 고유한 폴더를 가리킵니다.
          show_progress_bar=True) # 진행 바는 다시 켜도 좋습니다.

# (NEW) 종료 시간 기록 및 총 시간 계산
end_time = time.time()
total_time = end_time - start_time

# (NEW) ★최종 출력 메시지 수정
print(f"\n모델 학습 완료! (최고 성능 모델이 Google Drive '{model_save_path}' 경로에 저장됨)")

# (NEW) 5. 학습 결과(시간)를 가시성 좋게 출력
print("\n--- 🚀 학습 최종 요약 ---")
print(f"  총 학습 시간  : {total_time:.2f} 초")
print("  (학습 중 출력된 INFO 로그로 성능 향상을 확인하세요)")
print("------------------------")

총 스텝 수: 4220, Warmup 스텝 수: 211 (5%~)
모델 파인튜닝 시작... (Epochs: 5)
★★★ (중요) 최고 성능 모델이 Google Drive 경로에 저장됩니다: /content/gdrive/MyDrive/Colab_Notebooks/ssu_NLP/Trained_simcse_model/model_251114_0548 ★★★
검증자(evaluator)가 감지되었습니다. 매 에포크(Epoch) 종료 시 검증을 수행합니다.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/sentence_transformers/util/tensor.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  a = torch.tensor(a)



모델 학습 완료! (최고 성능 모델이 Google Drive '/content/gdrive/MyDrive/Colab_Notebooks/ssu_NLP/Trained_simcse_model/model_251114_0548' 경로에 저장됨)

--- 🚀 학습 최종 요약 ---
  총 학습 시간  : 3739.88 초
  (학습 중 출력된 INFO 로그로 성능 향상을 확인하세요)
------------------------


## 4단계: 모델 평가 및 임계값(Threshold) 설정
학습된 모델(혹은 저장된 모델)을 사용하여 '정상'과 '낚시성'을 얼마나 잘 구분하는지 테스트합니다.

In [ ]:
#-- 4.1. 학습된 모델로 유사도 계산 테스트 (★ 3.7셀 실행 직후 사용 ★)

# --- [ ★★★ 중요 ★★★ ] ---
# 이 셀은 3.7 셀 실행 '직후'에 런타임이 재시작되지 않았을 때만 작동합니다.
# 3.7 셀에서 생성된 'model_save_path' 변수를 그대로 사용합니다.
try:
    # (model_save_path는 3.7 셀에서 정의됨)
    print(f"3.7 셀에서 학습한 모델을 로드합니다: {model_save_path}")
    model = SentenceTransformer(model_save_path)
    print(f"저장된 모델 '{model_save_path}' 로드 완료")
except NameError:
     print(f"--- 🔴 오류: 'model_save_path' 변수를 찾을 수 없습니다. ---")
     print("런타임이 재시작된 것 같습니다. 노트북 상단의 [셀 4.1 (B) 수동 로드] 셀을 대신 사용하세요.")
except Exception as e:
    print(f"모델 로드 오류: {e}. Google Drive 경로를 확인하세요.")


# --- [ ★★★ (NEW) 더미 데이터 수정 ★★★ ] ---
# (핵심 요약문: "김봉진 대표가 매각설을 '부인'하고 '아시아 독자 진출'을 선언했다.")
test_summary_text = "김봉진 우아한형제들 대표가 매각설을 일축하며, 아시아 11개국 시장에 독자적으로 진출하겠다는 사업 계획을 밝혔다."
# --- [ ★★★ 수정 끝 ★★★ ] ---


# --- (NEW) 다양한 테스트 케이스 정의 ---

# 테스트 1: [정상] 의미가 거의 일치하는 Pair (Paraphrase)
# (요약문의 핵심 단어(부인, 독자 진출)를 동의어로 표현)
test_headline_1 = "우아한형제들, '매각설 사실무근'… 김봉진 대표 '아시아 독자 공략'"
test_summary_1 = test_summary_text

# 테스트 2: [낚시성] 의미가 정반대인 Pair (Contradiction)
# (주제어는 같지만, '일축' -> '인정'으로 핵심 의미가 정반대)
test_headline_2 = "김봉진 대표, 결국 매각 '공식 인정'… 'M&A 불가피했다'"
test_summary_2 = test_summary_text

# (NEW) 테스트 3: [낚시성] 주제는 비슷하나 핵심 내용이 없는 Pair (Vague Clickbait)
# (매각설, 김봉진 등 키워드는 있으나 '일축/인정' 여부가 없음)
test_headline_3 = "김봉진 대표 '충격 선언'... 수조원대 매각설의 진실은?"
test_summary_3 = test_summary_text

# (NEW) 테스트 4: [무관] 주제가 완전히 다른 Pair (Unrelated)
# (주제어 자체가 완전히 다름)
test_headline_4 = "주말부터 장마전선 북상, 전국에 폭우 쏟아질 듯"
test_summary_4 = test_summary_text

# --- [ ★★★ 수정 끝 ★★★ ] ---


# --- 각 문장을 임베딩 벡터로 변환 ---
print("테스트 케이스 임베딩 중...")
try:
    embedding_summary = model.encode(test_summary_text, convert_to_tensor=True) # 요약문은 한 번만 인코딩

    embedding1 = model.encode(test_headline_1, convert_to_tensor=True)
    embedding2 = model.encode(test_headline_2, convert_to_tensor=True)
    embedding3 = model.encode(test_headline_3, convert_to_tensor=True)
    embedding4 = model.encode(test_headline_4, convert_to_tensor=True) # (NEW) 4번째 임베딩


    # --- 코사인 유사도 계산 ---
    cos_sim_1 = util.cos_sim(embedding1, embedding_summary)[0][0]
    cos_sim_2 = util.cos_sim(embedding2, embedding_summary)[0][0]
    cos_sim_3 = util.cos_sim(embedding3, embedding_summary)[0][0]
    cos_sim_4 = util.cos_sim(embedding4, embedding_summary)[0][0] # (NEW) 4번째 유사도


    print("\n--- 유사도 계산 결과 ---")
    print(f"[테스트 1: 정상 (의미 일치)] (기대값: 매우 높음) \t 유사도: {cos_sim_1:.4f}")
    print(f"[테스트 2: 낚시성 (의미 반대)] (기대값: 낮음) \t 유사도: {cos_sim_2:.4f}")
    print(f"[테스트 3: 낚시성 (애매함)] (기대값: 중간/낮음) \t 유사도: {cos_sim_3:.4f}")
    print(f"[테스트 4: 무관 (주제 다름)] (기대값: 매우 낮음) \t 유사도: {cos_sim_4:.4f}")

except Exception as e:
    print(f"--- 🔴 오류 ---")
    print(f"임베딩 또는 유사도 계산 중 오류 발생: {e}")
    print("모델이 정상적으로 로드되었는지 확인하세요. (런타임 재시작 여부 확인)")
    # 오류 발생 시 이후 4.3 셀에서 NameError가 나지 않도록 임의값 할당
    cos_sim_1, cos_sim_2 = 0.0, 0.0
    cos_sim_3 = 0.0
    cos_sim_4 = 0.0 # (NEW) 4번째 변수 추가

3.7 셀에서 학습한 모델을 로드합니다: /content/gdrive/MyDrive/Colab_Notebooks/ssu_NLP/Trained_simcse_model/model_251114_0548
저장된 모델 '/content/gdrive/MyDrive/Colab_Notebooks/ssu_NLP/Trained_simcse_model/model_251114_0548' 로드 완료
테스트 케이스 임베딩 중...

--- 유사도 계산 결과 ---
[테스트 1: 정상 (의미 일치)] (기대값: 매우 높음) 	 유사도: 0.9604
[테스트 2: 낚시성 (의미 반대)] (기대값: 낮음) 	 유사도: 0.9459
[테스트 3: 낚시성 (애매함)] (기대값: 중간/낮음) 	 유사도: 0.9402
[테스트 4: 무관 (주제 다름)] (기대값: 매우 낮음) 	 유사도: 0.8916


In [ ]:
#-- 4.1. 학습된 모델로 유사도 계산 테스트 (★런타임 재시작 시 수동 로드★)
'''
import os # (os 임포트 추가)

# --- [ ★★★ 수정 필요 ★★★ ] ---
# 1. Google Drive의 'Trained_simcse_model' 폴더에서
#    불러오고 싶은 모델의 폴더명(예: 'model_251112_1841')을 복사합니다.
MANUAL_MODEL_NAME = "model_251113_1109" # ★★★ 여기에 테스트할 모델 폴더명 입력 ★★★

# 2. (NEW) 최종 경로를 완성합니다. (띄어쓰기 없는 경로 확인)
base_save_dir = "/content/gdrive/MyDrive/Colab_Notebooks/ssu_NLP/Trained_simcse_model"

# (중요) 4.2, 4.3 셀과의 호환성을 위해 변수명을 'model_save_path'로 유지합니다.
model_save_path = os.path.join(base_save_dir, MANUAL_MODEL_NAME)
# --- [ ★★★ 수정 끝 ★★★ ] ---

try:
    print(f"Google Drive에서 수동으로 모델을 로드합니다: {model_save_path}")
    model = SentenceTransformer(model_save_path)
    print(f"저장된 모델 '{model_save_path}' 로드 완료")
except Exception as e:
    print(f"모델 로드 오류: {e}")
    print("경로와 폴더명이 올바른지, Google Drive가 마운트되었는지 확인하세요.")


# --- [ ★★★ (NEW) 더미 데이터 수정 ★★★ ] ---
# (기준이 되는 핵심 요약문)
test_summary_text = "김봉진 우아한형제들 대표가 매각설을 일축하며, 아시아 11개국 시장에 독자적으로 진출하겠다는 사업 계획을 밝혔다."
# --- [ ★★★ 수정 끝 ★★★ ] ---


# --- (NEW) 다양한 테스트 케이스 정의 ---

# 테스트 1: [정상] 의미가 거의 일치하는 Pair (Paraphrase)
# (요약문의 핵심 단어(부인, 독자 진출)를 동의어로 표현)
test_headline_1 = "김봉진 대표, '매각설 일축'… 아시아 독자 진출 사업계획 발표"

# 테스트 2: [낚시성] 의미가 정반대인 Pair (Contradiction)
# (주제어는 같지만, '일축' -> '인정'으로 핵심 의미가 정반대)
test_headline_2 = "김봉진 대표, 결국 매각 '공식 인정'… 'M&A 불가피했다'"

# 테스트 3: [낚시성] 주제는 비슷하나 핵심 내용이 없는 Pair (Vague Clickbait)
# (매각설, 김봉진 등 키워드는 있으나 '일축/인정' 여부가 없음)
test_headline_3 = "김봉진 대표 '충격 선언'... 수조원대 매각설의 진실은?"

# 테스트 4: [무관] 주제가 완전히 다른 Pair (Unrelated)
# (주제어 자체가 완전히 다름)
test_headline_4 = "주말부터 장마전선 북상, 전국에 폭우 쏟아질 듯"

# --- [ ★★★ 수정 끝 ★★★ ] ---


# --- 각 문장을 임베딩 벡터로 변환 ---
print("테스트 케이스 임베딩 중...")
try:
    # (NEW) 요약문은 한 번만 인코딩하여 재사용
    embedding_summary = model.encode(test_summary_text, convert_to_tensor=True)

    embedding1 = model.encode(test_headline_1, convert_to_tensor=True)
    embedding2 = model.encode(test_headline_2, convert_to_tensor=True)
    embedding3 = model.encode(test_headline_3, convert_to_tensor=True)
    embedding4 = model.encode(test_headline_4, convert_to_tensor=True)


    # --- 코사인 유사도 계산 ---
    cos_sim_1 = util.cos_sim(embedding1, embedding_summary)[0][0]
    cos_sim_2 = util.cos_sim(embedding2, embedding_summary)[0][0]
    cos_sim_3 = util.cos_sim(embedding3, embedding_summary)[0][0]
    cos_sim_4 = util.cos_sim(embedding4, embedding_summary)[0][0]


    print("\n--- 유사도 계산 결과 ---")
    print(f"[테스트 1: 정상 (의미 일치)] (기대값: 매우 높음) \t 유사도: {cos_sim_1:.4f}")
    print(f"[테스트 2: 낚시성 (의미 반대)] (기대값: 낮음) \t 유사도: {cos_sim_2:.4f}")
    print(f"[테스트 3: 낚시성 (애매함)] (기대값: 중간/낮음) \t 유사도: {cos_sim_3:.4f}")
    print(f"[테스트 4: 무관 (주제 다름)] (기대값: 매우 낮음) \t 유사도: {cos_sim_4:.4f}")

except Exception as e:
    print(f"--- 🔴 오류 ---")
    print(f"임베딩 또는 유사도 계산 중 오류 발생: {e}")
    print("모델이 정상적으로 로드되었는지 확인하세요.")
    # 오류 발생 시 이후 4.3 셀에서 NameError가 나지 않도록 임의값 할당
    cos_sim_1, cos_sim_2 = 0.0, 0.0
    cos_sim_3, cos_sim_4 = 0.0, 0.0

'''

'\nimport os # (os 임포트 추가)\n\n# --- [ ★★★ 수정 필요 ★★★ ] ---\n# 1. Google Drive의 \'Trained_simcse_model\' 폴더에서\n#    불러오고 싶은 모델의 폴더명(예: \'model_251112_1841\')을 복사합니다.\nMANUAL_MODEL_NAME = "model_251113_1109" # ★★★ 여기에 테스트할 모델 폴더명 입력 ★★★\n\n# 2. (NEW) 최종 경로를 완성합니다. (띄어쓰기 없는 경로 확인)\nbase_save_dir = "/content/gdrive/MyDrive/Colab_Notebooks/ssu_NLP/Trained_simcse_model"\n\n# (중요) 4.2, 4.3 셀과의 호환성을 위해 변수명을 \'model_save_path\'로 유지합니다.\nmodel_save_path = os.path.join(base_save_dir, MANUAL_MODEL_NAME)\n# --- [ ★★★ 수정 끝 ★★★ ] ---\n\ntry:\n    print(f"Google Drive에서 수동으로 모델을 로드합니다: {model_save_path}")\n    model = SentenceTransformer(model_save_path)\n    print(f"저장된 모델 \'{model_save_path}\' 로드 완료")\nexcept Exception as e:\n    print(f"모델 로드 오류: {e}")\n    print("경로와 폴더명이 올바른지, Google Drive가 마운트되었는지 확인하세요.")\n\n\n# --- [ ★★★ (NEW) 더미 데이터 수정 ★★★ ] ---\n# (기준이 되는 핵심 요약문)\ntest_summary_text = "김봉진 우아한형제들 대표가 매각설을 일축하며, 아시아 11개국 시장에 독자적으로 진출하겠다는 사업 계획을 밝혔다."\n# --- [ ★★★ 수정 끝 ★★★ ] ---\n\n\n# --- (NEW)

In [ ]:
# 4.2. 최적의 임계값(Threshold) 찾기 (실제 검증 데이터 사용)

from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# --- 1. 실제 평가(Test) 데이터셋 준비 ---
# (NEW) 8번 셀(3.3)에서 분리한 10%의 'validation_data_list'를 사용합니다.

if 'validation_data_list' in locals() and validation_data_list:
    print(f"--- 1. {len(validation_data_list)}개의 실제 검증(validation) 데이터로 임계값 탐색 ---")

    # (NEW) 데이터를 test_pairs와 ground_truth_labels로 변환
    test_pairs = [(data[0], data[1]) for data in validation_data_list]
    ground_truth_labels = [data[2] for data in validation_data_list]

else:
    print("--- 1. 🔴 경고: 'validation_data_list'를 찾을 수 없습니다. (8번 셀 실행 필요) ---")
    print("--- 임시 더미 데이터로 대신 실행합니다. ---")
    test_pairs = [
        ("김봉진, 40억달러 딜로 아시아 11개국 총괄", "우아한형제들 김봉진 대표가..."),
        ("서울 아파트 거래량 3개월 연속 감소", "금리 인상과 대출 규제의 영향으로..."),
        ("김봉진 대표, 회사 매각 후 돌연 잠적", "우아한형제들 김봉진 대표가..."),
        ("서울 아파트 '폭락' 시작됐다!", "금리 인상과 대출 규제의 영향으로...")
    ]
    ground_truth_labels = [1, 1, 0, 0]


# --- 2. 모델로 유사도 계산 ---
# (model 변수는 3.7에서 학습된 모델을 사용)
# (6번 셀에서 임포트한 'util' (from sentence_transformers import util)을 사용)
print("--- 2. 임베딩 및 코사인 유사도 계산 중... (데이터가 많으면 시간이 걸릴 수 있습니다) ---")

# (NEW) 배치 인코딩을 사용하여 속도 향상
test_embeddings1 = model.encode([pair[0] for pair in test_pairs], convert_to_tensor=True, show_progress_bar=True)
test_embeddings2 = model.encode([pair[1] for pair in test_pairs], convert_to_tensor=True, show_progress_bar=True)

similarities = []
for emb1, emb2 in zip(test_embeddings1, test_embeddings2):
    sim = util.cos_sim(emb1, emb2)
    similarities.append(sim.item())

print(f"유사도 계산 완료. (샘플 5개: {np.round(similarities[:5], 4)})")

# --- 3. 최적의 임계값 탐색 (F1-Score 기준) ---

print("--- 3. 최적의 F1-Score 기준 임계값 탐색 중... ---")

best_threshold = 0
best_f1 = 0
best_acc = 0 # (NEW) F1이 최대일 때의 정확도

# 0.01부터 1.00까지 100개의 임계값 후보를 테스트
for threshold_candidate in [i * 0.01 for i in range(1, 101)]:
    # 현재 임계값을 기준으로 예측 (유사도가 임계값보다 크면 1, 아니면 0)
    preds = [1 if s > threshold_candidate else 0 for s in similarities]

    # F1-Score 계산
    f1 = f1_score(ground_truth_labels, preds, pos_label=1, zero_division=0)

    # 최고 F1-Score 갱신
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold_candidate
        best_acc = accuracy_score(ground_truth_labels, preds) # (NEW)

print("\n--- 4. 탐색 결과 (검증 데이터 10%) ---")
print(f"  최적의 F1-Score: {best_f1:.4f}")
print(f"  당시의 Accuracy : {best_acc:.4f}")
print(f"  최적의 임계값(Threshold): {best_threshold:.2f}")
print("\n(이 임계값을 5단계 워크플로우에 적용합니다.)")

--- 1. 3001개의 실제 검증(validation) 데이터로 임계값 탐색 ---
--- 2. 임베딩 및 코사인 유사도 계산 중... (데이터가 많으면 시간이 걸릴 수 있습니다) ---


Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

유사도 계산 완료. (샘플 5개: [0.665  0.6133 0.9579 0.9979 0.9   ])
--- 3. 최적의 F1-Score 기준 임계값 탐색 중... ---

--- 4. 탐색 결과 (검증 데이터 10%) ---
  최적의 F1-Score: 0.8472
  당시의 Accuracy : 0.7927
  최적의 임계값(Threshold): 0.73

(이 임계값을 5단계 워크플로우에 적용합니다.)


In [ ]:
# 4.3. (수동 테스트) 계산된 최적 임계값 적용

# (NEW) 4.2 셀에서 계산된 'best_threshold' 변수를 사용합니다.
try:
    THRESHOLD = best_threshold
    print(f"--- 4.2에서 찾은 최적 임계값(Threshold) {THRESHOLD:.2f} 적용 ---")
except NameError:
    print("--- 🔴 오류: 'best_threshold' 변수가 없습니다. (4.2 셀 실행 필요) ---")
    print("--- 임시 임계값 0.65를 사용합니다. ---")
    THRESHOLD = 0.65

# (4.1 셀의 'cos_sim_1' ~ 'cos_sim_4' 변수를 사용합니다)
# (4.1 셀을 실행해야 이 변수들이 존재합니다)
try:
    print("\n(4.1 셀의 더미 데이터로 테스트)")

    # 테스트 1 판정
    judgment_1 = '[정상]' if cos_sim_1 > THRESHOLD else '[낚시성]'
    print(f"[테스트 1: 정상 (의미 일치)] (유사도 {cos_sim_1:.4f}) \t 판정: {judgment_1}")

    # 테스트 2 판정
    judgment_2 = '[정상]' if cos_sim_2 > THRESHOLD else '[낚시성]'
    print(f"[테스트 2: 낚시성 (의미 반대)] (유사도 {cos_sim_2:.4f}) \t 판정: {judgment_2}")

    # (NEW) 테스트 3 판정
    judgment_3 = '[정상]' if cos_sim_3 > THRESHOLD else '[낚시성]'
    print(f"[테스트 3: 낚시성 (애매함)] (유사도 {cos_sim_3:.4f}) \t 판정: {judgment_3}")

    # (NEW) 테스트 4 판정
    judgment_4 = '[정상]' if cos_sim_4 > THRESHOLD else '[낚시성]'
    print(f"[테스트 4: 무관 (주제 다름)] (유사도 {cos_sim_4:.4f}) \t 판정: {judgment_4}")

except NameError:
     print("\n--- 🔴 경고: 'cos_sim_...' 변수가 없습니다. (4.1 셀 실행 필요) ---")

--- 4.2에서 찾은 최적 임계값(Threshold) 0.73 적용 ---

(4.1 셀의 더미 데이터로 테스트)
[테스트 1: 정상 (의미 일치)] (유사도 0.9604) 	 판정: [정상]
[테스트 2: 낚시성 (의미 반대)] (유사도 0.9459) 	 판정: [정상]
[테스트 3: 낚시성 (애매함)] (유사도 0.9402) 	 판정: [정상]
[테스트 4: 무관 (주제 다름)] (유사도 0.8916) 	 판정: [정상]


## 5단계: 최종 워크플로우 적용 (개념)
이제 완성된 두 모델(KoBART, KoSimCSE)을 사용자의 최종 워크플로우 2.C에 적용합니다.
1. 입력: 새로운 기사 [헤드라인 + 본문]이 입력됩니다.
2. 요약: [본문]을 파인튜닝된 KoBART 모델에 입력하여 [생성된 요약문]을 얻습니다.
3. 임베딩: [헤드라인]과 [생성된 요약문]을 파인튜닝된 KoSimCSE 모델(model.encode)에 넣어 두 개의 임베딩 벡터를 추출합니다.
4. 유사도 계산: 두 벡터의 코사인 유사도(util.cos_sim)를 계산합니다.
5. 판별: 이 유사도 점수를 4단계에서 찾은 **임계값(THRESHOLD)**과 비교하여 '정상'/'낚시성' 여부를 판별합니다.
6. 업데이트: 판별 결과를 바탕으로 해당 언론사의 신뢰도 점수를 업데이트합니다.